<a href="https://colab.research.google.com/github/amkutuzova/analyze_sentiment/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

def analyze_sentiment(review):
    inputs = tokenizer(review, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

df=pd.read_csv('/content/cafe_reviews.csv')
df['predicted_sentiment'] = df['text'].apply(analyze_sentiment)

sentiment_labels = {0: "Negative", 1: "Neutral", 2: "Positive"}

sentiment_counts = df['predicted_sentiment'].value_counts().sort_index()
labels = [sentiment_labels[int(idx)] for idx in sentiment_counts.index]

plt.bar(labels, sentiment_counts, color=['gray', 'green', 'red'])
plt.xlabel('Тональность')
plt.ylabel('Количество отзывов')
plt.title('Распределение тональности отзывов (Predicted)')
plt.show()